In [ ]:
import mne
import mne_bids
from mne_bids import BIDSPath
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt
import numpy as np
from mne.minimum_norm import apply_inverse, make_inverse_operator

from nilearn import *

from nih2mne.utilities.bids_helpers import get_mri_dict
n_jobs = 10 #Number of parrallel operations

from mne.viz._brain.view import views_dicts
from mne.viz import set_3d_view
from PIL import Image
def get_image_npy(fig_):
    _=fig_.canvas.draw()
    rgb = fig_.canvas.tostring_rgb()
    width, height = fig_.canvas.get_width_height()
    img = Image.frombytes('RGB', (width, height), rgb)
    return np.array(img)

## Set some generic BIDS information

In [ ]:
#Raw data
bids_root = op.join('/data/', os.environ['USER'], 'meg_workshop_data')  
#Processed Data Folder: contains (freesurfer / Day1 / Day2) 
deriv_root = op.join(bids_root, 'derivatives')  
#This course data - bids derivatives outputs for Day2 - hey thats today
project_root = op.join(deriv_root, 'Day2')  
#Freesurfer brain surface reconstruction
fs_subjects_dir = subjects_dir = op.join(deriv_root, 'freesurfer','subjects')

### Find data for a subject

In [ ]:
subject = 'ON02811'
data_dict = get_mri_dict(subject,bids_root, task='airpuff', project='Day2')

# get_mri_dict is a helper function that provides all of the loaders for the mri integration
# Each dictionary item codes for the specific loader of that file - see use in next cell of code
data_dict

#### Load the preprocessed data - Items from Lab3 MRI Processing

In [ ]:
bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
src = fwd['src']
trans = data_dict['trans'].load()

In [ ]:
#Load the Raw data
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
raw.filter(0.3, 110, n_jobs=n_jobs)

### Prepare Imaging for Source Localization

In [ ]:
evts, evtsid = mne.events_from_annotations(raw)

In [ ]:
epochs = mne.Epochs(raw, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)

In [ ]:
evk_stim = epochs['stim'].average()
evk_mssingstim = epochs['missingstim'].average()

In [ ]:
%matplotlib inline
_=evk_stim.plot()

In [ ]:
cov = mne.compute_covariance(epochs['stim'], tmin=0, tmax = 0.1, method='shrunk', cv=5, n_jobs=n_jobs)

In [ ]:
evk_stim.crop(0.0,0.06)

In [ ]:
%matplotlib inline
_= evk_stim.plot_topomap(times=np.arange(0, 0.05, 0.005))

In [ ]:
dip = mne.fit_dipole(evk_stim, cov, bem, trans=trans, n_jobs = n_jobs, rank='full')

In [ ]:
_ = dip[0].plot_locations(trans, 'sub-'+subject, fs_subjects_dir, mode="orthoview")

## If we filter by goodness of fit, we see there is less dispersion of the dipoles

In [ ]:
_=dip[0][dip[0].gof>20].plot_locations(trans, 'sub-'+subject, fs_subjects_dir, mode="orthoview", color='g')

### What if we take the dipole with the best goodness of fit and use that as a fixed dipole

In [ ]:
#From https://mne.tools/stable/auto_tutorials/inverse/20_dipole_fit.html#sphx-glr-auto-tutorials-inverse-20-dipole-fit-py
best_idx = np.argmax(dip[0].gof)
best_time = dip[0].times[best_idx]
dip_fixed = mne.fit_dipole(
    evk_stim,
    cov,
    bem,
    trans,
    pos=dip[0].pos[best_idx],
    ori=dip[0].ori[best_idx],
)[0]

### Notice the difference between free and fixed dipoles.  Free Dipoles will never be negative because they will rotate to point in the direction of the estimated current.  The fixed dipole also has cleaner parameters - due to the "best" estimated location and orientaiton over the whole timeseries

In [ ]:
fig, axes = plt.subplots(2,2)
times=dip[0].times
axes[0,0].plot(times, dip[0].amplitude, 'g')
axes[0,0].set_ylabel('Amplitude')
axes[0,0].set_title('Moving Dipole')
axes[1,0].plot(times, dip[0].gof, 'g')
axes[1,0].set_ylabel('GOF')
axes[0,1].plot(times, dip_fixed.data[0,:])
axes[0,1].set_title('Fixed Dipole')
axes[1,1].plot(times, dip_fixed.data[1,:])
fig.tight_layout()

# MNE solution

## MNE solution is in current/Amps

In [ ]:
inv = make_inverse_operator(evk_stim.info, fwd, cov, loose=0.0, depth=0.8, verbose=True)
snr = 3.0
lambda2 = 1.0 / snr**2
stc = abs(apply_inverse(evk_stim, inv, lambda2, "MNE", verbose=True))

### This is normally an interactive window - but because of biowulf, we are casting this to an image to plot

In [ ]:
maxval=stc._data.max()
kwargs = dict(
    initial_time=0.04,
    hemi="lh",
    subjects_dir=subjects_dir,
    size=(600, 600),
    clim=dict(kind="values", lims=[.5*maxval, .7*maxval, .9*maxval]),
    smoothing_steps=7,
    surface='white'
)
brain = stc.plot(**kwargs) 
brain.add_text(0.1, 0.9, "MNE", "title", font_size=14)

In [ ]:
%matplotlib inline
set_3d_view(brain,**views_dicts['lh']['parietal'])
mne_img_parietal=brain.plotter.screenshot()
set_3d_view(brain,**views_dicts['lh']['lateral'])
mne_img_lateral=brain.plotter.screenshot()
# brain.plotter.close()
plt.subplot(1,2,1)
plt.imshow(mne_img_parietal)
plt.subplot(1,2,2)
plt.imshow(mne_img_lateral)
plt.show()

## dSPM solution is divided by projected noise -- This is a statistic

In [ ]:
inv = make_inverse_operator(evk_stim.info, fwd, cov, loose=0.0, depth=0.8, verbose=True)
snr = 3.0
lambda2 = 1.0 / snr**2
dspm_stc = abs(apply_inverse(evk_stim, inv, lambda2, "dSPM", verbose=True))

In [ ]:
maxval=dspm_stc._data.max()
kwargs = dict(
    initial_time=0.04,
    hemi="lh",
    subjects_dir=subjects_dir,
    size=(600, 600),
    clim=dict(kind="values", lims=[.5*maxval, .7*maxval, .9*maxval]),
    smoothing_steps=7,
    surface='white', 
)
brain = dspm_stc.plot(figure=1, **kwargs)
brain.add_text(0.1, 0.9, "dSPM", "title", font_size=14)

In [ ]:
%matplotlib inline
set_3d_view(brain,**views_dicts['lh']['parietal'])
dspm_img_parietal=brain.plotter.screenshot()
set_3d_view(brain,**views_dicts['lh']['lateral'])
dspm_img_lateral=brain.plotter.screenshot()
# brain.plotter.close()
plt.subplot(1,2,1)
plt.imshow(dspm_img_parietal)
plt.subplot(1,2,2)
plt.imshow(dspm_img_lateral)
plt.show()

### Comparison between MNE and dSPM

In [ ]:
plt.subplot(1,2,1)
plt.imshow(mne_img_parietal)
plt.subplot(1,2,2)
plt.imshow(dspm_img_parietal)
plt.suptitle('MNE (left) and dSPM (right) - both at 50/70/90 thresholds of their global time-series max value')
plt.tight_layout()